In [1]:
import pandas as pd
import math
import numpy as np
import scipy.stats as stats
import plotly as py
import plotly.graph_objs as go
import plotly.express as px

incidence_data = pd.read_excel('../raw_data/australia/aihw-can-122-cancer_incidence.xlsx')
population_data = pd.read_csv('../population_data/USCB_Australia.csv', header=1, sep=r'\s*,\s*', engine='python')

In [2]:
codes = incidence_data['ICD10 codes'].unique()
actual_incidence = incidence_data[incidence_data['Data type'] == 'Actual'].reset_index(drop = True)

for code in codes:
    try:
        code = code
        code_df = actual_incidence[actual_incidence['ICD10 codes'] == code]
        gender = 'Males'
        population_gender = 'Male'
        age_group = 'All ages'
        population_ages = 'Total'

        gender_incidence = code_df[code_df['Sex'] == gender]

        gender_allages_incidence_df = gender_incidence[gender_incidence['Age group (years)'] == age_group]

        select_years_rates = gender_allages_incidence_df.iloc[27:].reset_index(drop = True)
        rates = select_years_rates['Age-specific rate\n(per 100,000)']
        gender_population_select_years = population_data.loc[population_data['Age'] == population_ages].iloc[0:8][f'{population_gender} Population'].reset_index(drop = True)
        cases = [math.floor(rates[i] / 100000 * gender_population_select_years[i]) for i in range(len(gender_population_select_years))]

        rates = np.array(gender_allages_incidence_df['Age-specific rate\n(per 100,000)']).astype(float)
        years = np.array(gender_allages_incidence_df['Year']).astype(float)

        slope, intercept, r_value, p_value, std_err = stats.linregress(years,rates)

        fig = px.line(x=years, y=rates, title=f'Australia, {code} Incidence, {gender}, {age_group}')

        fig.update_layout(yaxis=dict(range=[0,max(rates) + 1]))

        fig.update_layout(
            xaxis_title="Year",
            yaxis_title="Incidence (Cases per 100,000)",
            plot_bgcolor='rgba(0,0,0,0)',
            autosize=False,
                width=800,
                height=600)

        #fig.show()
        fig.write_image(f'plots/incidence/incidence_{code}_{gender}_{age_group}.png')
        
    except IndexError:
        print(f'Index Error for {code}')
        
print('Process Completed')

Index Error for C81–C96, D45, D46, D47.1, D47.3–D47.5
Index Error for C53
Index Error for C51–C58
Index Error for D46
Index Error for C44
Index Error for C56
Index Error for C54–C55
Index Error for C52
Index Error for C51
Index Error for nan
Process Completed
